# Import Model

In [1]:
import warnings

import pandas as pd

warnings.simplefilter("ignore")

In [2]:
!pip install accelerate loralib bitsandbytes datasets git+https://github.com/huggingface/peft.git transformers sentencepiece

  Cloning https://github.com/huggingface/peft.git to /private/var/folders/jk/1l9p3qzn5lj9r717c1wy8f9h0000gn/T/pip-req-build-nwjho_67
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /private/var/folders/jk/1l9p3qzn5lj9r717c1wy8f9h0000gn/T/pip-req-build-nwjho_67
  Resolved https://github.com/huggingface/peft.git to commit 3714aa2fff158fdfa637b2b65952580801d890b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import os
import sys
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
from peft import PeftModel
from typing import List


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/dastlab01/Desktop/KDDChallenge2023/venvllama/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /Users/dastlab01/Desktop/KDDChallenge2023/venvllama/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/dastlab01/Desktop/KDDChallenge2023/venvllama/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/dastlab01/Desktop/KDDChallenge2023/venvllama/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/dastlab01/Desktop/KDDChallenge2023/venvllama/lib/python3.9/site-packages/bitsa

In [4]:
torch.autocast("cpu")

# LLaMATokenizer is the old class name, no unexpected tokenization will be produced using LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    torch_dtype=torch.float32,
    device_map="auto",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

# Generation Test


In [5]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        {instruction}

        ### Input:
        {input}
        ### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction:
        {instruction}
        ### Response:"""




In [6]:
generation_config = GenerationConfig(
    temperature=1.0,
    top_p=1,
    top_k=40,
    num_beams=4
)

In [7]:
instruction = "From the description of a rule: identify the 'trigger', identify the 'action', write a IF 'trigger' THEN 'action' rule."
 
input = "When Foobot detects pollution, the system should turn on the appliance plugged into your WeMo Switch."

prompt = generate_prompt(instruction=instruction, input=input)
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
input_ids = inputs["input_ids"]

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    output_scores=True,
    max_new_tokens=256
)


KeyboardInterrupt



In [ ]:
output = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0].split("### Response:")[1].strip()
print(output)
print("IF Foobot Pollution is high THEN WeMo Smart Plug Turn on")

In [ ]:
trigger = output.split("THEN")[0].strip().replace('IF','').replace('(','').replace(')','').split('AND')
action = output.split("THEN")[1].strip().replace('(','').replace(')','').split('AND')
print(trigger)
print(action)

# Prompter

In [ ]:
"""
A dedicated helper to manage templates and prompt building.
"""

import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = "alpacal.json"
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()


# Fine Tuning

In [ ]:
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"  # Allow batched inference

In [ ]:
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt


In [ ]:
# model/data params
base_model: str = "decapoda-research/llama-7b-hf"  # the only required argument
data_path: str =  'AntonioRenatoMontefusco/kddChallenge2023'
output_dir: str = "./Output"
# training hyperparams
batch_size: int = 128
micro_batch_size: int = 8
num_epochs: int = 1
learning_rate: float = 3e-4
cutoff_len: int = 512
val_set_size: int = 1000
# lora hyperparams
lora_r: int = 16
lora_alpha: int = 16
lora_dropout: float = 0.05
lora_target_modules: List[str] = ["q_proj","k_proj","v_proj","o_proj"]
# llm hyperparams
train_on_inputs: bool = True  # if False masks out inputs in loss
add_eos_token: bool = False
group_by_length: bool = False  # faster but produces an odd training loss curve
# wandb params
wandb_project: str = ""
wandb_run_name: str = ""
wandb_watch: str = ""  # options: false | gradients | all
wandb_log_model: str = ""  # options: false | true
resume_from_checkpoint: str = None  # either training checkpoint or final adapter
prompt_template_name: str = "alpaca"  # The prompt template to use will default to alpaca.

In [ ]:
gradient_accumulation_steps = batch_size // micro_batch_size

prompter = Prompter(prompt_template_name)

device_map = "auto"
world_size = int(os.environ.get("WORLD_SIZE", 1))
ddp = world_size != 1
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    gradient_accumulation_steps = gradient_accumulation_steps // world_size

# Check if parameter passed or if set within environ
use_wandb = len(wandb_project) > 0 or (
    "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
)
# Only overwrite environ if wandb param passed
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
if len(wandb_watch) > 0:
    os.environ["WANDB_WATCH"] = wandb_watch
if len(wandb_log_model) > 0:
    os.environ["WANDB_LOG_MODEL"] = wandb_log_model

In [ ]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

In [ ]:
model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

In [ ]:
!pip install zstandard
!pip install psutil

In [ ]:
from datasets import load_dataset
import psutil


In [ ]:
import random

data_path = "output"
data_files = {"train": f"output/second_half.json"}
data = {}
data["train"] = load_dataset("json", data_files="output/second_half.json")

if val_set_size > 0:
    train_val = []
    for example in data["train"]:
        train_val.append(example)
        if len(train_val) >= val_set_size:
            break

    random.shuffle(train_val)
    train_data = train_val[val_set_size:]
    val_data = train_val[:val_set_size]
else:
    train_data = data["train"]
    val_data = None

random.shuffle(train_data)

In [ ]:
trainer = transformers.Trainer(
          model=model,
          train_dataset=train_data,
          eval_dataset=val_data,
          args=transformers.TrainingArguments(
              per_device_train_batch_size=micro_batch_size,
              gradient_accumulation_steps=gradient_accumulation_steps,
              warmup_steps=100,
              num_train_epochs=num_epochs,
              learning_rate=learning_rate,
              fp16=False,
              logging_steps=10,
              optim="adamw_torch",
              evaluation_strategy="steps" if val_set_size > 0 else "no",
              save_strategy="steps",
              eval_steps=200 if val_set_size > 0 else None,
              save_steps=200,
              output_dir=output_dir,
              save_total_limit=3,
              load_best_model_at_end=True if val_set_size > 0 else False,
              ddp_find_unused_parameters=False if ddp else None,
              group_by_length=group_by_length,
              report_to="wandb" if use_wandb else None,
              run_name=wandb_run_name if use_wandb else None,
          ),
          data_collator=transformers.DataCollatorForSeq2Seq(
              tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
          ),
      )

In [ ]:
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

In [ ]:
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

In [ ]:
model.save_pretrained(output_dir)

print(
    "\n If there's a warning about missing keys above, please disregard :)"
)